In [3]:
#Regression Algorithm 

def get_data(url, drop=[]):
  import pandas as pd
  df = pd.read_csv(url)
  if len(drop) > 0:
    for col in drop:
      df.drop(columns=[col], inplace=True)
  return df

def bin_groups(df, percent=.05):
  import pandas as pd
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      for group, count in df[col].value_counts().iteritems():
        if count / len(df) < percent:
          df.loc[df[col] == group, col] = 'Other'
  return df

def drop_columns_missing_50(df, cutoff=.5):
  import pandas as pd
  for col in df:
    if df[col].isna().sum() / len(df) > cutoff:
      df.drop(columns=[col], inplace=True)
  return df

def impute_mean(df):
  from sklearn.impute import SimpleImputer
  import pandas as pd, numpy as np
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      df = pd.get_dummies(df, columns=[col], drop_first=True)
  imp = SimpleImputer(missing_values=np.nan, strategy='mean')
  df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
  return df

def fs_variance(df, label="", p=0.8):
  from sklearn.feature_selection import VarianceThreshold
  import pandas as pd

  if label != "":
    X = df.drop(columns=[label])
    
  sel = VarianceThreshold(threshold=(p * (1 - p)))
  sel.fit_transform(X)

  # Add the label back in after removing poor features
  return df[sel.get_feature_names_out()].join(df[label])

def fit_crossvalidate_mlr(df, k, label, repeat=True):
  from sklearn.linear_model import LinearRegression
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  import pandas as pd
  from numpy import mean, std
  X = df.drop(label,axis=1)
  y = df[label]
  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=cv, n_jobs=-1)
  print(f'Average R-squared:\t{mean(scores)}')
  return LinearRegression().fit(X, y)

def dump_pickle(model, file_name):
  import pickle
  pickle.dump(model, open(file_name, "wb"))

def load_pickle(file_name):
  import pickle
  model = pickle.load(open(file_name, "rb"))
  return model

In [4]:
#train and test set

df = get_data('CuperCut_TrainCrash.csv', ["CRASH_ID"])
df = bin_groups(df, 0.05)
df = drop_columns_missing_50(df)
df = impute_mean(df)

model = fit_crossvalidate_mlr(df, 10, label='CRASH_SEVERITY_ID')

Average R-squared:	0.18930810923360947


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('CuperCut_TrainCrash.csv')

for col in df:
  if not pd.api.types.is_numeric_dtype(df[col]):
    df = df.join(pd.get_dummies(df[col], prefix=col))

y = df['CRASH_SEVERITY_ID']
X = df.drop(columns=["CRASH_SEVERITY_ID"])
X = X.select_dtypes(np.number)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=1)
X_test.head()

,CRASH_ID,MILEPOINT,CRASH_DATETIME_01/01/2016 01:00:00 AM,CRASH_DATETIME_01/01/2016 01:04:00 AM,CRASH_DATETIME_01/01/2016 01:12:00 AM,CRASH_DATETIME_01/01/2016 01:25:00 AM,CRASH_DATETIME_01/01/2016 01:57:00 AM,CRASH_DATETIME_01/01/2016 02:00:00 AM,CRASH_DATETIME_01/01/2016 02:31:00 AM,CRASH_DATETIME_01/01/2016 02:35:00 AM,...,COUNTY_NAME_SUMMIT,COUNTY_NAME_TOOELE,COUNTY_NAME_UINTAH,COUNTY_NAME_UTAH,COUNTY_NAME_WASATCH,COUNTY_NAME_WASHINGTON,COUNTY_NAME_WAYNE,COUNTY_NAME_WEBER,WORK_ZONE_RELATED_False,WORK_ZONE_RELATED_True
1433,10823501,0.100000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4880,10835904,2.063000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
17786,10915400,6.148000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11580,10893333,46.942987,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1699,10816864,0.100000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
print(len(X_test))
print(len(X_train))

17563
1951


In [9]:
x_test = get_data('CuperCut_TrainCrash.csv', ['CRASH_ID'])
x_test = bin_groups(x_test, 0.05)
x_test = drop_columns_missing_50(x_test)
x_test = impute_mean(x_test)


model = fit_crossvalidate_mlr(x_test, 10, label='CRASH_SEVERITY_ID')

Average R-squared:	0.18930810923360947


In [8]:
print(len(x_test))

19514


In [10]:
# What does it mean to get a negative R-squared value? https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#:~:text=(coefficient%20of%20determination)%20regression%20score,get%20a%20score%20of%200.0.
# Why is our model so much worse than random? Is there a pattern between the important features?
# MEME time

def vif(df, label=""):
  import pandas as pd
  from sklearn.linear_model import LinearRegression
  
  # initialize dictionaries
  vif_dict, tolerance_dict = {}, {}

  # drop unnecessary columns if they are found in the dataframe
  if label in df.columns: df.drop(columns=[label], inplace=True)
  if 'const' in df.columns: df.drop(columns=['const'], inplace=True)

  # form input data for each exogenous variable
  for col in df:
    y = df[col]
    X = df.drop(columns=[col])
    
    # extract r-squared from the fit
    r_squared = LinearRegression().fit(X, y).score(X, y)

    # calculate VIF
    if r_squared < 1: # Prevent division by zero runtime error
      vif = 1/(1 - r_squared) 
    else:
      vif = 100
    vif_dict[col] = vif

    # calculate tolerance
    tolerance = 1 - r_squared
    tolerance_dict[col] = tolerance

  # generate the DataFrame to return
  return pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict}).sort_values(by=['VIF'], ascending=False)

vif(x_test).head(20)


,VIF,Tolerance
ROUTE_Other,6.407352,0.156071
SINGLE_VEHICLE,4.271322,0.234120
MAIN_ROAD_NAME_Other,4.220854,0.236919
MILEPOINT,3.998592,0.250088
COUNTY_NAME_SALT LAKE,3.490743,0.286472
ROADWAY_DEPARTURE,3.193965,0.313091
COUNTY_NAME_Other,3.142115,0.318257
ROUTE_89,3.028699,0.330175
CITY_Other,2.535910,0.394336
COUNTY_NAME_UTAH,2.232440,0.447940


In [11]:
# Because of the multicollinearity issues we're seeing with our data, let's fit a decision tree instead of MLR

# Multivariate feature importance for tree model
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from numpy import mean

y = x_test.CRASH_SEVERITY_ID
X = x_test.drop(columns=['CRASH_SEVERITY_ID'])

model = DecisionTreeRegressor()
model.fit(X, y)


cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=12345)
scores = cross_val_score(DecisionTreeRegressor(), X, y, scoring='r2', cv=cv, n_jobs=-1)
print(f'Decision Tree R-squared:\t{mean(scores)}')

Decision Tree R-squared:	-0.5408340276487364


In [14]:
df_fi = pd.DataFrame({"Feature Importance":model.feature_importances_}, index=x_test.drop(columns=['CRASH_SEVERITY_ID']).columns)
df_fi.sort_values(by=['Feature Importance'], ascending=False).head(20)

df_fi

,Feature Importance
MILEPOINT,0.511498
PEDESTRIAN_INVOLVED,0.039664
BICYCLIST_INVOLVED,0.022246
MOTORCYCLE_INVOLVED,0.055615
IMPROPER_RESTRAINT,0.008152
UNRESTRAINED,0.015195
DUI,0.011498
INTERSECTION_RELATED,0.013694
WILD_ANIMAL_RELATED,0.004807
DOMESTIC_ANIMAL_RELATED,0.000966


In [11]:
# Let's look at Linear Models (top-down) and test many of the Linear Models (bottom-up) to see which model produces the best fit metrics
# Scikit-learn documenation: https://scikit-learn.org/stable/modules/linear_model.html#linear-model

fit = {}    # Use this to store each of the fit metrics
models = {} # Use this to store each of the models
        
# 1. LINEAR MODELS: Assumes normal distribution, homoscedasticity, no multicollinearity, independence, and no auto-correlation (some exceptions apply; some of these algorithms are better at handling violations of these assumptions).
import sklearn.linear_model as lm, pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean

# We will automate this later, just create
X = x_test.drop(columns=['CRASH_SEVERITY_ID'])
y = x_test['CRASH_SEVERITY_ID']

# Set up a standard cross_validation object to use for each algorithm
cv = KFold(n_splits=5, random_state=12345, shuffle=True)

# 1.1. Ordinary Least Squares Multiple Linear Regression
model_ols = lm.LinearRegression()
fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['OLS'] = model_ols

# 1.2. Ridge Regression: More robust to multicollinearity
model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Ridge'] = model_rr

# 1.3. Lasso Regression: Better for sparse values like RetweetCount where most are zeros but a few have many retweets.
model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Lasso'] = model_lr

# 1.4. Least Angle Regression: Good when the number of features is greater than the number of samples.
model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['LARS'] = model_llr

# 1.5. Bayesian Regression: Probability based and allows regularization parameters, automatically tuned to data.
model_br = lm.BayesianRidge()
fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Bayesian'] = model_br

# 1.6. Generalized Linear Regression (Poisson): Good for non-normal distribution, count-based data, and a Poisson distribution.
model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Poisson'] = model_pr

# 1.7. Generalized Linear Regression (Gamma): Good for non-normal distribution, continuous data, and a Gamma distribution.
model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Gamma'] = model_gr

# 1.8. Generalized Linear Regression (Inverse Gamma): Good non-normal distribution, continuous data, and an inverse Gamma distribution.
model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Inverse'] = model_igr


# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

,R-squared
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Poisson,0.019274
Lasso,0.000182
Gamma,-0.000095
Inverse,-0.000095
LARS,-0.000095


In [12]:
#support vector machines futhers the research of which model looks better
# Let's continue to test another family of regression algorithms: Support Vector Machines

# SUPPORT VECTOR MACHINES: Ideal for noisy data with large gaps among values
from sklearn import svm

# 1.9. SVM: this is the default SVM, parameters can be modified to make this more accurate
model_svm = svm.SVR()
fit['SupportVM'] = mean(cross_val_score(model_svm, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['SupportVM'] = model_svm

# 1.10. Linear SVM: Faster than SVM but only considers a linear model
model_lsvm = svm.LinearSVR()
fit['Linear SVM'] = mean(cross_val_score(model_lsvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Linear SVM'] = model_lsvm

# 1.11. NuSVM: 
model_nusvm = svm.NuSVR()
fit['NuSupportVM'] = mean(cross_val_score(model_nusvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['NuSupportVM'] = model_nusvm

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

#The Ridge is still the best set of data


,R-squared
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Poisson,0.019274
Lasso,0.000182
Gamma,-0.000095
Inverse,-0.000095
LARS,-0.000095
NuSupportVM,-0.004918
Linear SVM,-0.062214


In [13]:
# Next, we will try the k-nearest neighbors (KNN) algorithm, which can be used for both labeled 
# and unlabeled data as well as both regression and classification problems

# KNN: NEAREST NEIGHBORS REGRESSION
from sklearn import neighbors

# 1.12. KNeighborsRegressor: 
model_knnr = neighbors.KNeighborsRegressor(n_neighbors=10, weights='uniform')
fit['KNNeighbors'] = mean(cross_val_score(model_knnr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['KNNeighbors'] = model_knnr

# 1.13. KNeighborsRegressor: 
model_knnrd = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
fit['KNNeighborsD'] = mean(cross_val_score(model_knnrd, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['KNNeighborsD'] = model_knnrd

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

,R-squared
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Poisson,0.019274
KNNeighbors,0.017739
Lasso,0.000182
Gamma,-0.000095
Inverse,-0.000095
LARS,-0.000095
NuSupportVM,-0.004918


In [14]:
# Next, let’s try a single algorithm called a Gaussian process regression (GPR). 
# GPR has some great benefits, including more accuracy with small datasets

# GAUSSIAN PROCESS REGRESSION
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

# 1.14. GaussianProcessRegressor:
model_gpr = gaussian_process.GaussianProcessRegressor(DotProduct() + WhiteKernel())
fit['GaussianP'] = mean(cross_val_score(model_gpr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['GaussianP'] = model_gpr

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\gaussian_process\_gpr.py", line 272, in fit
    self._constrained_optimization(
  File "C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn

,R-squared
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Poisson,0.019274
KNNeighbors,0.017739
Lasso,0.000182
Gamma,-0.000095
Inverse,-0.000095
LARS,-0.000095
NuSupportVM,-0.004918


In [15]:
# DECISION TREE MODELS: no assumptions about the data
import sklearn.tree as tree
import sklearn.ensemble as se

# 1.15. Decision Tree Regression
model_dt = tree.DecisionTreeRegressor(random_state=12345)
fit['Dec Tree'] = mean(cross_val_score(model_dt, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Dec Tree'] = model_dt


# DECISION TREE-BASED ENSEMBLE MODELS: great for minimizing overfitting, these are based on averaging many unique sub-samples and combining algorithms 
# 1.16. Decision Forrest
model_df = se.RandomForestRegressor(random_state=12345)
fit['Dec Forest'] = mean(cross_val_score(model_df, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Dec Forest'] = model_df

# 1.17. ExtraTreesRegressor
model_etr = se.ExtraTreesRegressor(random_state=12345)
fit['Extra Trees'] = mean(cross_val_score(model_etr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Extra Trees'] = model_etr

# 1.18. AdaBoostRegressor
model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
fit['AdaBoost DT'] = mean(cross_val_score(model_abr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['AdaBoost DT'] = model_abr

# 1.19. GradientBoostingRegressor
model_gbr = se.GradientBoostingRegressor(random_state=12345)
fit['Grad. Boost'] = mean(cross_val_score(model_gbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Grad. Boost'] = model_gbr

# 1.20. HistGradientBoostingRegressor
model_hgbr = se.HistGradientBoostingRegressor(random_state=12345)
fit['HG Boost'] = mean(cross_val_score(model_hgbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['HG Boost'] = model_hgbr

# 1.21. VotingRegressor: will combine other algorithms into an average; kind of cool
model_vr = se.VotingRegressor(estimators=[('DT', model_dt), ('DF', model_df), ('ETR', model_etr), ('ABR', model_abr), ('GBR', model_gbr)])
fit['Voting'] = mean(cross_val_score(model_vr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Voting'] = model_vr

# 1.22. StackingRegressor
from sklearn.linear_model import RidgeCV, LassoCV
estimators = [('ridge', RidgeCV()), ('lasso', LassoCV(random_state=42)), ('svr', svm.SVR(C=1, gamma=1e-6))]
model_sr = se.StackingRegressor(estimators=estimators, final_estimator=se.GradientBoostingRegressor(random_state=12345))
fit['Stacking'] = mean(cross_val_score(model_sr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Stacking'] = model_sr

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

,R-squared
HG Boost,0.196775
Grad. Boost,0.195872
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Stacking,0.178878
Voting,0.079599
Dec Forest,0.036783
Poisson,0.019274
KNNeighbors,0.017739


In [16]:
# Since we just tried a gradient boosted model and had great success, it is worth taking a slight pause from the scikit-learn package 
# to add an algorithm from a popular package known as XGBoost. XGBoost is built on top of the scikit-learn package and works much the same way. 
# It is popular because it offers a faster and often slightly more accurate version of a gradient boosting algorithm.

from xgboost import XGBRegressor

# 1.23. XGBRegressor
model_xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['XGBoost'] = model_xgb

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


,R-squared
HG Boost,0.196775
Grad. Boost,0.195872
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Stacking,0.178878
Voting,0.079599
Dec Forest,0.036783
XGBoost,0.035160
Poisson,0.019274


In [17]:
# NEURAL-NETWORK MODELS: Based on deep learning methods
import sklearn.neural_network as nn

# 1.23. MLPRegressor
model_nn = nn.MLPRegressor(max_iter=1000, random_state=12345) # Turn max_iter way up or down to get a more accurate result
fit['NeuralNet'] = mean(cross_val_score(model_nn, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['NeuralNet'] = model_nn

# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

,R-squared
HG Boost,0.196775
Grad. Boost,0.195872
Ridge,0.188900
OLS,0.188899
Bayesian,0.188888
Stacking,0.178878
Voting,0.079599
Dec Forest,0.036783
XGBoost,0.035160
Poisson,0.019274


In [18]:
# Take a few minutes to create a function that will automate the process that we just went through. It should
# (1) Try every algorithm and 
# (2) Automatically select the best one
# Do this with the first three algorithms first, then we can expand it to include all algorithms
# This function will replace fit_crossvalidate_mlr() in our pipeline (so that function is a good starting point)

def fit_crossvalidate_reg(df, label, k=10, r=5, repeat=True):
  import sklearn.linear_model as lm
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  import pandas as pd
  from numpy import mean, std
  X = df.drop(label,axis=1)
  y = df[label]
  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)

  fit = {}
  model = {}

  # Create the model objects
  
  model_ols = lm.LinearRegression()
  model_rr = lm.Ridge(alpha=0.5)
  model_lr = lm.Lasso(alpha=0.1)

  # Fit a cross-validated R squared score and add it to the dict
  
  fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))

  # Add the model to another dict; make sure the keys have the same names as the list above  

  models['OLS'] = model_ols
  models['Ridge'] = model_rr
  models['Lasso'] = model_lr

  df_fit = pd.DataFrame({'R-squared':fit})
  df_fit.sort_values(by=['R-squared'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)

In [19]:
df = get_data('CuperCut_TrainCrash.csv', ['CRASH_ID'])
df = bin_groups(df)
df = drop_columns_missing_50(df)
df = impute_mean(df)

# Feature selection and modeling pipeline
df = fs_variance(df, label="CRASH_SEVERITY_ID", p=.5)
model = fit_crossvalidate_reg(df, 'CRASH_SEVERITY_ID', 5, 2)

# Deploy/store the model
dump_pickle(model, 'best_reg_model.sav')

       R-squared
Ridge  -0.000343
OLS    -0.000343
Lasso  -0.000344


In [20]:
def fit_crossvalidate_reg(df, label, k=10, r=5, repeat=True):
  import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  from numpy import mean, std
  from sklearn import svm
  from sklearn import gaussian_process
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from xgboost import XGBRegressor

  X = df.drop(columns=[label])
  y = df[label]

  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  
  fit = {}    # Use this to store each of the fit metrics
  models = {} # Use this to store each of the models

  # Create the model objects
  model_ols = lm.LinearRegression()
  model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
  model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
  model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
  model_br = lm.BayesianRidge()
  model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
  model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
  model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
  model_svm = svm.SVR()
  model_lsvm = svm.LinearSVR()
  model_nusvm = svm.NuSVR()
  model_gpr = gaussian_process.GaussianProcessRegressor(DotProduct() + WhiteKernel())
  model_df = se.RandomForestRegressor(random_state=12345)
  model_etr = se.ExtraTreesRegressor(random_state=12345)
  model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
  model_gbr = se.GradientBoostingRegressor(random_state=12345)
  model_hgbr = se.HistGradientBoostingRegressor(random_state=12345)
  model_vr = se.VotingRegressor(estimators=[('DF', model_df), ('ETR', model_etr), ('ABR', model_abr), ('GBR', model_gbr)])
  estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=42)), ('svr', svm.SVR(C=1, gamma=1e-6))]
  model_sr = se.StackingRegressor(estimators=estimators, final_estimator=se.GradientBoostingRegressor(random_state=12345))
  model_xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
  
  # Fit a crss-validated R squared score and add it to the dict
  fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['SupportVM'] = mean(cross_val_score(model_svm, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Linear SVM'] = mean(cross_val_score(model_lsvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['NuSupportVM'] = mean(cross_val_score(model_nusvm, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['GaussianP'] = mean(cross_val_score(model_gpr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Dec Forest'] = mean(cross_val_score(model_df, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Extra Trees'] = mean(cross_val_score(model_etr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['AdaBoost DT'] = mean(cross_val_score(model_abr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Grad. Boost'] = mean(cross_val_score(model_gbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['HG Boost'] = mean(cross_val_score(model_hgbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Voting'] = mean(cross_val_score(model_vr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['Stacking'] = mean(cross_val_score(model_sr, X, y, scoring='r2', cv=cv, n_jobs=-1))
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='r2', cv=cv, n_jobs=-1))

  # Add the model to another dict; make sure the keys have the same names as the list above
  models['OLS'] = model_ols
  models['Ridge'] = model_rr
  models['Lasso'] = model_lr
  models['LARS'] = model_llr
  models['Bayesian'] = model_br
  models['Poisson'] = model_pr
  models['Gamma'] = model_gr
  models['Inverse'] = model_igr
  models['SupportVM'] = model_svm
  models['Linear SVM'] = model_lsvm
  models['NuSupportVM'] = model_nusvm
  models['GaussianP'] = model_gpr
  models['Dec Forest'] = model_df
  models['Extra Trees'] = model_etr
  models['AdaBoost DT'] = model_abr
  models['Grad. Boost'] = model_gbr
  models['HG Boost'] = model_hgbr
  models['Voting'] = model_vr
  models['Stacking'] = model_sr
  models['XGBoost'] = model_xgb

  # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
  df_fit = pd.DataFrame({'R-squared':fit})
  df_fit.sort_values(by=['R-squared'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)

# Now, see how this fits into the pipeline:---------------------------------------------
# Data cleaning and preparation pipeline
df = get_data('CuperCut_TrainCrash.csv', ['CRASH_ID'])
df = bin_groups(df)
df = drop_columns_missing_50(df)
df = impute_mean(df)

# Feature selection and modeling pipeline
df = fs_variance(df, label="CRASH_SEVERITY_ID", p=.5)
model = fit_crossvalidate_reg(df, 'CRASH_SEVERITY_ID', 5, 2)

# Deploy/store the model
dump_pickle(model, 'best_reg_model.sav')

C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\gaussian_process\_gpr.py", line 272, in fit
    self._constrained_optimization(
  File "C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklea

             R-squared
Grad. Boost   0.002416
HG Boost      0.002353
Poisson      -0.000340
Bayesian     -0.000343
Ridge        -0.000343
OLS          -0.000343
Lasso        -0.000344
Gamma        -0.000628
Inverse      -0.000628
LARS         -0.000628
NuSupportVM  -0.007027
Stacking     -0.025756
Voting       -0.114816
XGBoost      -0.183408
SupportVM    -0.190952
AdaBoost DT  -0.294332
Linear SVM   -0.318005
Dec Forest   -0.340857
Extra Trees  -0.497498
GaussianP          NaN


In [26]:
#CLASSIFICATION ALOGORITHMS
# Classification models are those which predict a categorical dependent variable
# Let's look at a similar function to fit_crossvalidate_reg() but for classification algorithms
# Note some of the differences (e.g. model objects, scoring) and some of the similarities 

def fit_crossvalidate_clf(df, label, k=10, r=5, repeat=True):
  import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se, numpy as np
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  from numpy import mean, std
  from sklearn import svm
  from sklearn import gaussian_process
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
  from sklearn import svm
  from sklearn.naive_bayes import CategoricalNB
  from xgboost import XGBClassifier
  from sklearn import preprocessing
  from sklearn.neural_network import MLPClassifier

  X = df.drop(columns=[label])
  y = df[label]

  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  
  fit = {}    # Use this to store each of the fit metrics
  models = {} # Use this to store each of the models
  
  # Create the model objects
  model_log = lm.LogisticRegression(max_iter=100)
  model_logcv = lm.RidgeClassifier()
  model_sgd = lm.SGDClassifier(max_iter=1000, tol=1e-3)
  model_pa = lm.PassiveAggressiveClassifier(max_iter=1000, random_state=12345, tol=1e-3)
  model_per = lm.Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
  model_knn = KNeighborsClassifier(n_neighbors=3)
  model_svm = svm.SVC(decision_function_shape='ovo') # Remove the parameter for two-class model
  model_nb = CategoricalNB()
  model_bag = se.BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
  model_ada = se.AdaBoostClassifier(n_estimators=100, random_state=12345)
  model_ext = se.ExtraTreesClassifier(n_estimators=100, random_state=12345)
  model_rf = se.RandomForestClassifier(n_estimators=10)
  model_hgb = se.HistGradientBoostingClassifier(max_iter=100)
  model_vot = se.VotingClassifier(estimators=[('lr', model_log), ('rf', model_ext), ('gnb', model_hgb)], voting='hard')
  model_gb = se.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=12345)), ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))]
  final_estimator = se.GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=12345)
  model_st = se.StackingRegressor(estimators=estimators, final_estimator=final_estimator)
  model_xgb = XGBClassifier()
  model_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=12345)

  # Fit a cross-validated R squared score and add it to the dict
  fit['Logistic'] = mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_logcv, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SGD'] = mean(cross_val_score(model_sgd, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['PassiveAggressive'] = mean(cross_val_score(model_pa, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Perceptron'] = mean(cross_val_score(model_per, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['KNN'] = mean(cross_val_score(model_knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SVM'] = mean(cross_val_score(model_svm, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NaiveBayes'] = mean(cross_val_score(model_nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Bagging'] = mean(cross_val_score(model_bag, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['AdaBoost'] = mean(cross_val_score(model_ada, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['ExtraTrees'] = mean(cross_val_score(model_ext, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['RandomForest'] = mean(cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['HistGradient'] = mean(cross_val_score(model_hgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Voting'] = mean(cross_val_score(model_vot, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['GradBoost'] = mean(cross_val_score(model_gb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NeuralN'] = mean(cross_val_score(model_nn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))

  # Add the model to another dictionary; make sure the keys have the same names as the list above
  models['Logistic'] = model_log
  models['Ridge'] = model_logcv
  models['SGD'] = model_sgd
  models['PassiveAggressive'] = model_pa
  models['Perceptron'] = model_per
  models['KNN'] = model_knn
  models['SVM'] = model_svm
  models['NaiveBayes'] = model_nb
  models['Bagging'] = model_bag
  models['AdaBoost'] = model_ada
  models['ExtraTrees'] = model_ext
  models['RandomForest'] = model_rf
  models['HistGradient'] = model_hgb
  models['Voting'] = model_vot
  models['GradBoost'] = model_gb
  models['XGBoost'] = model_xgb
  models['NeuralN'] = model_nn

  # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
  df_fit = pd.DataFrame({'Accuracy':fit})
  df_fit.sort_values(by=['Accuracy'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)

In [28]:
# Data cleaning and preparation pipeline
df = get_data('CuperCut_TrainCrash.csv', ['CRASH_ID'])
df = bin_groups(df)
df = drop_columns_missing_50(df)

# Drop the label so it does not get dummy-coded, then join it back in after
df = impute_mean(df.drop(columns=["CRASH_SEVERITY_ID"])).join(df.CRASH_SEVERITY_ID)

# Feature selection and modeling pipeline
df = fs_variance(df, label="CRASH_SEVERITY_ID", p=.5)
model = fit_crossvalidate_clf(df, "CRASH_SEVERITY_ID", 5, 2)

# Deployment pipeline
dump_pickle(model, 'best_clf_model.sav')

                   Accuracy
AdaBoost           0.706621
HistGradient       0.706621
Logistic           0.706621
NeuralN            0.706621
Ridge              0.706621
SVM                0.706621
Voting             0.706621
GradBoost          0.706416
XGBoost            0.703136
Bagging            0.691478
Perceptron         0.650331
KNN                0.647689
SGD                0.626195
ExtraTrees         0.601824
RandomForest       0.593394
PassiveAggressive  0.551496
NaiveBayes              NaN


In [29]:
# In both the fit_crossvalidate_reg() and fit_crossvalidate_clf() functions, we are testing
# all of the algorithms with a specific set of parameters..
# (note: this concept can be applied to any algorithm)

df = get_data('CuperCut_TrainCrash.csv', ['CRASH_ID'])
df = bin_groups(df)
df = drop_columns_missing_50(df)
df = impute_mean(df)
df = fs_variance(df, label="CRASH_SEVERITY_ID", p=.5)

In [30]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, GridSearchCV

params = {
    "booster": ['gbtree', 'gblinear', 'dart'],
    "learning_rate": [0.1, 0.3, 0.5], 
    "objective": ['reg:squarederror'], 
}

# Create the hypertuning search object
model_xgb = GridSearchCV(
    XGBRegressor(), 
    params, 
    n_jobs=-1, # Number of threads to use; -1 means use all available
    scoring='r2', # Options: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    cv=KFold(n_splits=3), # Choose any type of cross_validation you want
    verbose=2, # How much information to display in the results; options: 1, 2, or 3
    refit=True # This saves the best-fitting model
    )

model_xgb.fit(df.drop(columns=['CRASH_SEVERITY_ID']), df.CRASH_SEVERITY_ID)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_wei...
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    sub

In [31]:
print(f'Best parameters: {model_xgb.best_params_}')
print(f'R-squared:\t {model_xgb.best_score_}')

print(f'All results:')
pd.DataFrame(model_xgb.cv_results_)

Best parameters: {'booster': 'gblinear', 'learning_rate': 0.1, 'objective': 'reg:squarederror'}
R-squared:	 -0.0050218722741327175
All results:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_learning_rate,param_objective,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.033247,0.076533,0.009520,0.001173,gbtree,0.1,reg:squarederror,"{'booster': 'gbtree', 'learning_rate': 0.1, 'o...",-0.004882,-0.017937,-0.007432,-0.010084,0.005650,5
1,1.028746,0.048612,0.011256,0.002159,gbtree,0.3,reg:squarederror,"{'booster': 'gbtree', 'learning_rate': 0.3, 'o...",-0.038542,-0.047962,-0.042925,-0.043143,0.003849,6
2,1.048402,0.123369,0.009998,0.004405,gbtree,0.5,reg:squarederror,"{'booster': 'gbtree', 'learning_rate': 0.5, 'o...",-0.082128,-0.065993,-0.088139,-0.078753,0.009350,9
3,0.272891,0.024977,0.005368,0.000483,gblinear,0.1,reg:squarederror,"{'booster': 'gblinear', 'learning_rate': 0.1, ...",-0.004119,-0.009710,-0.001237,-0.005022,0.003518,1
4,0.224862,0.018369,0.005913,0.000852,gblinear,0.3,reg:squarederror,"{'booster': 'gblinear', 'learning_rate': 0.3, ...",-0.004357,-0.009366,-0.001367,-0.005030,0.003300,2
5,0.202708,0.002308,0.005506,0.000357,gblinear,0.5,reg:squarederror,"{'booster': 'gblinear', 'learning_rate': 0.5, ...",-0.004357,-0.009366,-0.001367,-0.005030,0.003300,3
6,4.287942,0.019862,0.103524,0.002509,dart,0.1,reg:squarederror,"{'booster': 'dart', 'learning_rate': 0.1, 'obj...",-0.004882,-0.017937,-0.007432,-0.010084,0.005650,4
7,4.323068,0.013203,0.071453,0.011080,dart,0.3,reg:squarederror,"{'booster': 'dart', 'learning_rate': 0.3, 'obj...",-0.038542,-0.047962,-0.042925,-0.043143,0.003849,7
8,3.073420,1.457672,0.045416,0.011562,dart,0.5,reg:squarederror,"{'booster': 'dart', 'learning_rate': 0.5, 'obj...",-0.082128,-0.065993,-0.088139,-0.078753,0.009350,8


In [33]:
# Halving is the process of training a selection of models on very small datasets to improve speed. 
# Then, much like a tournament, the best n models are selected and trained again using a slightly larger dataset. 
# The advantage over RandomizedSearchCV is that every model gets trained at least once. 
# The potential downside is that a very good hyperparameter set may simply happen to get a poor sample to 
# train with and end up being missed.

from sklearn.experimental import enable_halving_search_cv # Must import this first
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV

params = {
    "booster": ['gbtree', 'gblinear', 'dart'], # Default is gbtree
    "learning_rate": [0.1, 0.3, 0.5],  # It accepts float [0,1] specifying learning rate for training process. Default = 0.3
    "objective": ['reg:squarederror'], # List of possible values: https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
    "max_depth": [3, 6, 9], # Must be between 3-10; default = 6
    "min_child_weight": [1, 2, 3], # Default = 1
    "gamma": [0, 0.1, 0.2], # Default = 0
    "subsample": [0.8, 0.9, 1], # Default = 1
    "colsample_bytree": [0.8, 1], # Default = 1
    "alpha": [0, .001, 1, 100], # Default = 0
}

# Create the hypertuning object
model_xgb = HalvingRandomSearchCV( # If this takes to long, change it to HalvingRandomSearchCV
    XGBRegressor(), 
    params,
    factor=2, # The 'halving' parameter; proportion of candidates selected for each iteration
    n_candidates=32, # The number of hyperparameter value sets to randomly sample
    resource='n_estimators', # Default = n_samples, but use n_estimators for boosting algorithms
    n_jobs=-1, # Number of threads to use; -1 means use all available
    scoring='r2', # Options: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    cv=KFold(n_splits=3), # Choose any type of cross_validation you want
    verbose=2, # How much information to display in the results; options: 1, 2, or 3
    max_resources=800, # The maximum number of resources (either n_samples or n_estimators) to use in each round
    min_resources=100, # The maximum number of resources (either n_samples or n_estimators) to use in each round
    refit=True # This saves the best-fitting model
    )

model_xgb.fit(df.drop(columns=['CRASH_SEVERITY_ID']), df.CRASH_SEVERITY_ID)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 100
max_resources_: 800
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 32
n_resources: 100
Fitting 3 folds for each of 32 candidates, totalling 96 fits
----------
iter: 1
n_candidates: 16
n_resources: 200
Fitting 3 folds for each of 16 candidates, totalling 48 fits
----------
iter: 2
n_candidates: 8
n_resources: 400
Fitting 3 folds for each of 8 candidates, totalling 24 fits
----------
iter: 3
n_candidates: 4
n_resources: 800
Fitting 3 folds for each of 4 candidates, totalling 12 fits


C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


HalvingRandomSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                      estimator=XGBRegressor(base_score=None, booster=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None,
                                             enable_categorical=False,
                                             gamma=None, gpu_id=None,
                                             importance_type=None,
                                             interaction_constraints=None,
                                             learning_rate=None,
                                             max_delta_step=None,
                                             max_depth=None,
                                             min_...
                      factor=2, max_resources=800, min_resources=100,
                      n_candidates=32, n_jo

In [34]:
print(f'Best parameters: {model_xgb.best_params_}')
print(f'R-squared:\t {model_xgb.best_score_}')

print(f'All results:')
pd.DataFrame({
    "Parameters":model_xgb.cv_results_['params'], 
    "Mean Fit Score":model_xgb.cv_results_['mean_test_score'],
    "Std Fit Score":model_xgb.cv_results_['std_test_score']
})

Best parameters: {'subsample': 1, 'objective': 'reg:squarederror', 'min_child_weight': 2, 'max_depth': 6, 'learning_rate': 0.5, 'gamma': 0.2, 'colsample_bytree': 0.8, 'booster': 'dart', 'alpha': 100, 'n_estimators': 800}
R-squared:	 -0.001773257616376922
All results:


,Parameters,Mean Fit Score,Std Fit Score
0,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.005030,0.003300
1,"{'subsample': 0.9, 'objective': 'reg:squareder...",-0.001965,0.005567
2,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.001773,0.005798
3,"{'subsample': 0.9, 'objective': 'reg:squareder...",-0.005167,0.003485
4,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.005030,0.003300
5,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.098468,0.008246
6,"{'subsample': 0.9, 'objective': 'reg:squareder...",-0.005149,0.003365
7,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.020612,0.005194
8,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.010118,0.006795
9,"{'subsample': 0.8, 'objective': 'reg:squareder...",-0.005149,0.003365


In [35]:
# Randomized Parameter Optimization allows us to randomly select n parameter combinations
# What are some pros/cons of this approach?

from sklearn.model_selection import RandomizedSearchCV

params = {
    "booster": ['gbtree', 'gblinear', 'dart'], # Default is gbtree
    "learning_rate": [0.1, 0.3, 0.5],  # It accepts float [0,1] specifying learning rate for training process. Default = 0.3
    "objective": ['reg:squarederror'], # List of possible values: https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
    "max_depth": [3, 6, 9], # Must be between 3-10; default = 6
    "min_child_weight": [1, 2, 3], # Default = 1
    "gamma": [0, 0.1, 0.2], # Default = 0
    "subsample": [0.8, 0.9, 1], # Default = 1
    "colsample_bytree": [0.8, 1], # Default = 1
    "alpha": [0, .001, 1, 100], # Default = 0
}

# Create the hypertuning object
model_xgb = RandomizedSearchCV(
    XGBRegressor(), 
    params,
    n_iter=10, # Number of random samples to fit; default is 10
    n_jobs=-1, # Number of threads to use; -1 means use all available
    scoring='r2', # Options: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    cv=KFold(n_splits=3), # Choose any type of cross_validation you want
    verbose=2, # How much information to display in the results; options: 1, 2, or 3
    refit=True # This saves the best-fitting model
    )

model_xgb.fit(df.drop(columns=['CRASH_SEVERITY_ID']), df.CRASH_SEVERITY_ID)

print(f'Best parameters: {model_xgb.best_params_}')
print(f'R-squared:\t {model_xgb.best_score_}')

print(f'All results:')
pd.DataFrame({
    "Parameters":model_xgb.cv_results_['params'], 
    "Mean Fit Score":model_xgb.cv_results_['mean_test_score'],
    "Std Fit Score":model_xgb.cv_results_['std_test_score']
})

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\stefa\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Best parameters: {'subsample': 0.8, 'objective': 'reg:squarederror', 'min_child_weight': 3, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.8, 'booster': 'gbtree', 'alpha': 100}
R-squared:	 -0.0017696554330600318
All results:


,Parameters,Mean Fit Score,Std Fit Score
0,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.005361,0.003355
1,"{'subsample': 0.8, 'objective': 'reg:squareder...",-0.001770,0.005383
2,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.005361,0.003355
3,"{'subsample': 0.8, 'objective': 'reg:squareder...",-0.005022,0.003518
4,"{'subsample': 0.9, 'objective': 'reg:squareder...",-0.042550,0.003654
5,"{'subsample': 0.9, 'objective': 'reg:squareder...",-0.001965,0.005567
6,"{'subsample': 0.8, 'objective': 'reg:squareder...",-0.002392,0.004693
7,"{'subsample': 1, 'objective': 'reg:squarederro...",-0.016942,0.012636
8,"{'subsample': 0.9, 'objective': 'reg:squareder...",-0.002170,0.005267
9,"{'subsample': 0.8, 'objective': 'reg:squareder...",-0.057439,0.006133
